# **Inicializacion de Librerias y Nube.**

In [1]:
from google.colab import drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#Instalamos las dependencias necesarias en el servidor (pyspark, etc)
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install findspark
!pip install pyspark


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 33 kB/s 
     |████████████████████████████████| 198 kB 46.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=3bc33afe6ba2424a8101a8ebaeee1d1e3b1736f591e637167cd9fd2f453d1c65
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [3]:
#Importamos las librerías de PySpark
import os
import findspark
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import *
from pyspark.sql import SparkSession

#Establecemos las variables de ambiente
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"

#Inicializamos Spark
findspark.init()
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext

In [4]:
#Importamos todos los documentos en un DataFrame de Spark, imrpimimos el esquema

tweets_df=spark.read.csv('/content/drive/MyDrive/Proyecto integrador/Datasets/Raw/KaggleDS-NASDAQ/Tweet.csv',inferSchema=True,header=True,sep=',')


In [5]:
tweets_df.printSchema()

root
 |-- tweet_id: long (nullable = true)
 |-- writer: string (nullable = true)
 |-- post_date: integer (nullable = true)
 |-- body: string (nullable = true)
 |-- comment_num: string (nullable = true)
 |-- retweet_num: string (nullable = true)
 |-- like_num: string (nullable = true)



In [6]:
tweets_df.show(10)

+------------------+---------------+----------+--------------------+-----------+-----------+--------+
|          tweet_id|         writer| post_date|                body|comment_num|retweet_num|like_num|
+------------------+---------------+----------+--------------------+-----------+-----------+--------+
|550441509175443456|VisualStockRSRC|1420070457|lx21 made $10,008...|          0|          0|       1|
|550441672312512512|    KeralaGuy77|1420070496|Insanity of today...|          0|          0|       0|
|550441732014223360|    DozenStocks|1420070510|S&P100 #Stocks Pe...|          0|          0|       0|
|550442977802207232|   ShowDreamCar|1420070807|$GM $TSLA: Volksw...|          0|          0|       1|
|550443807834402816|   i_Know_First|1420071005|Swing Trading: Up...|          0|          0|       1|
|550443808606126081|  aaplstocknews|1420071005|Swing Trading: Up...|          0|          0|       1|
|550443809700851716|     iknowfirst|1420071005|Swing Trading: Up...|          0|  

In [7]:
from pyspark.sql.functions import udf,struct
from datetime import datetime

from pyspark.sql.functions import col
from pyspark.sql.types import StringType,BooleanType,DateType,IntegerType,ArrayType,StructType
from pyspark.sql.functions import when

import nltk
import pandas as pd
import matplotlib.pyplot as plt
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer


In [8]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('words')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.


True

# **Text Prep**

In [9]:
#def unix_utc_converter(unix_date):
#
#    print(unix_date)
#    token_date = datetime.utcfromtimestamp(unix_date).strftime('%Y-%m-%d %H:%M:%S') 

#    return 

#udf_datetime = udf(unix_utc_converter, ArrayType(StringType()))
#tweets_df2 = tweets_df.withColumn("post_date", udf_datetime(struct([tweets_df[x] for x in tweets_df.columns])))


date_udf = udf(lambda s: datetime.utcfromtimestamp(s).strftime('%Y-%m-%d'), StringType())
time_udf = udf(lambda s: datetime.utcfromtimestamp(s).strftime('%H:%M:%S'), StringType())

tweets_df2 = tweets_df.withColumn("post_date_utc", date_udf(col('post_date')))
tweets_df2 = tweets_df2.withColumn("post_time_utc", time_udf(col('post_date')))
tweets_df2.show(5)

#UNIX TIMESTAMP

#val df = sqlContext.sql("select from_unixtime(ts,'YYYY-MM-dd') as `ts` from mr")

+------------------+---------------+----------+--------------------+-----------+-----------+--------+-------------+-------------+
|          tweet_id|         writer| post_date|                body|comment_num|retweet_num|like_num|post_date_utc|post_time_utc|
+------------------+---------------+----------+--------------------+-----------+-----------+--------+-------------+-------------+
|550441509175443456|VisualStockRSRC|1420070457|lx21 made $10,008...|          0|          0|       1|   2015-01-01|     00:00:57|
|550441672312512512|    KeralaGuy77|1420070496|Insanity of today...|          0|          0|       0|   2015-01-01|     00:01:36|
|550441732014223360|    DozenStocks|1420070510|S&P100 #Stocks Pe...|          0|          0|       0|   2015-01-01|     00:01:50|
|550442977802207232|   ShowDreamCar|1420070807|$GM $TSLA: Volksw...|          0|          0|       1|   2015-01-01|     00:06:47|
|550443807834402816|   i_Know_First|1420071005|Swing Trading: Up...|          0|          

In [10]:
stop_words_nltk = set(stopwords.words('english'))
wordnet_lemmatizer = WordNetLemmatizer()
lancaster = LancasterStemmer()

def textprep(record):
    text  = record[3]
    tokens = text.split()
       
    tokens = [re.sub('[^a-zA-Z]','',word) for word in tokens]                                       
    tokens = [word.lower() for word in tokens if len(word)>2 and word.lower() not in stop_words_nltk]
    tokens = [wordnet_lemmatizer.lemmatize(w) for w in tokens]

    print(tokens)
    tokString = ' '.join([str(x) for x in tokens])
    #tokString = ' '.join(map(str, tokens))

    
    return tokens

udf_textprep = udf(textprep , ArrayType(StringType()))
df_tweets_clean = tweets_df2.withColumn("tokens", udf_textprep(struct([tweets_df2[x] for x in tweets_df2.columns])))
df_tweets_clean.show(5)


+------------------+---------------+----------+--------------------+-----------+-----------+--------+-------------+-------------+--------------------+
|          tweet_id|         writer| post_date|                body|comment_num|retweet_num|like_num|post_date_utc|post_time_utc|              tokens|
+------------------+---------------+----------+--------------------+-----------+-----------+--------+-------------+-------------+--------------------+
|550441509175443456|VisualStockRSRC|1420070457|lx21 made $10,008...|          0|          0|       1|   2015-01-01|     00:00:57|[made, aapl, chec...|
|550441672312512512|    KeralaGuy77|1420070496|Insanity of today...|          0|          0|       0|   2015-01-01|     00:01:36|[insanity, today,...|
|550441732014223360|    DozenStocks|1420070510|S&P100 #Stocks Pe...|          0|          0|       0|   2015-01-01|     00:01:50|[stock, performan...|
|550442977802207232|   ShowDreamCar|1420070807|$GM $TSLA: Volksw...|          0|          0|  

# **Analisis de Sentimientos**

## Instalación de librerias

In [11]:
pip install vaderSentiment

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 125 kB 5.3 MB/s 


In [12]:
pip install textBlob

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Inicialización de Librerias

In [13]:
import pandas as pd
import numpy as np

from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

from sklearn.metrics import accuracy_score


In [14]:
df_tweets_clean.show(15,False)

+------------------+---------------+----------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------+-----------+--------+-------------+-------------+---------------------------------------------------------------------------------------------------------------------------------------+
|tweet_id          |writer         |post_date |body                                                                                                                                                                    |comment_num|retweet_num|like_num|post_date_utc|post_time_utc|tokens                                                                                                                                 |
+------------------+---------------+----------+-----------------------------------------------------------------------------------------------------------------------------

# Análisis de Sentimientos TextBlob y Vader

In [15]:
def tb_sentiment(tokens):
  text = ' '.join([str(x) for x in tokens])
  sentiment = TextBlob(text).sentiment.polarity
  return sentiment

sentiment_udf = udf(lambda s: tb_sentiment(s), StringType())
df_tweets_clean = df_tweets_clean.withColumn("textBlob_sentiment", sentiment_udf(col('tokens')))
df_tweets_clean.show(20,False)

+------------------+---------------+----------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------+-----------+--------+-------------+-------------+---------------------------------------------------------------------------------------------------------------------------------------+-------------------+
|tweet_id          |writer         |post_date |body                                                                                                                                                                          |comment_num|retweet_num|like_num|post_date_utc|post_time_utc|tokens                                                                                                                                 |textBlob_sentiment |
+------------------+---------------+----------+-------------------------------------------------------------------------

In [16]:
vader = SentimentIntensityAnalyzer()
def vader_sentiment(tokens):
  text = ' '.join([str(x) for x in tokens])
  sentiment = vader.polarity_scores(text)
  return sentiment['compound']

sentiment_udf = udf(lambda s: vader_sentiment(s), StringType())
df_tweets_clean = df_tweets_clean.withColumn("vader_sentiment", sentiment_udf(col('tokens')))
df_tweets_clean.show(25,False)

+------------------+---------------+----------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------+-----------+--------+-------------+-------------+----------------------------------------------------------------------------------------------------------------------------------------------+-------------------+---------------+
|tweet_id          |writer         |post_date |body                                                                                                                                                                                           |comment_num|retweet_num|like_num|post_date_utc|post_time_utc|tokens                                                                                                                                        |textBlob_sentiment |vader_sentiment|
+------------------+---------------+----

In [17]:
df_tweets_clean = df_tweets_clean.drop('tokens')

In [18]:
df_tweets_clean.printSchema()

root
 |-- tweet_id: long (nullable = true)
 |-- writer: string (nullable = true)
 |-- post_date: integer (nullable = true)
 |-- body: string (nullable = true)
 |-- comment_num: string (nullable = true)
 |-- retweet_num: string (nullable = true)
 |-- like_num: string (nullable = true)
 |-- post_date_utc: string (nullable = true)
 |-- post_time_utc: string (nullable = true)
 |-- textBlob_sentiment: string (nullable = true)
 |-- vader_sentiment: string (nullable = true)



In [19]:
df_tweets_clean =  df_tweets_clean.withColumn("comment_num",col("comment_num").cast(IntegerType())) \
                                  .withColumn("retweet_num",col("retweet_num").cast(IntegerType())) \
                                  .withColumn("like_num",col("like_num").cast(IntegerType())) \
                                  .withColumn("post_date_utc",col("post_date_utc").cast(DateType())) \
                                  .withColumn("textBlob_sentiment",col("textBlob_sentiment").cast('float')) \
                                  .withColumn("vader_sentiment",col("vader_sentiment").cast('float')) 
                                  
df_tweets_clean.printSchema()

root
 |-- tweet_id: long (nullable = true)
 |-- writer: string (nullable = true)
 |-- post_date: integer (nullable = true)
 |-- body: string (nullable = true)
 |-- comment_num: integer (nullable = true)
 |-- retweet_num: integer (nullable = true)
 |-- like_num: integer (nullable = true)
 |-- post_date_utc: date (nullable = true)
 |-- post_time_utc: string (nullable = true)
 |-- textBlob_sentiment: float (nullable = true)
 |-- vader_sentiment: float (nullable = true)



In [20]:
df_tweets_clean.show(25)

+------------------+---------------+----------+--------------------+-----------+-----------+--------+-------------+-------------+------------------+---------------+
|          tweet_id|         writer| post_date|                body|comment_num|retweet_num|like_num|post_date_utc|post_time_utc|textBlob_sentiment|vader_sentiment|
+------------------+---------------+----------+--------------------+-----------+-----------+--------+-------------+-------------+------------------+---------------+
|550441509175443456|VisualStockRSRC|1420070457|lx21 made $10,008...|          0|          0|       1|   2015-01-01|     00:00:57|               0.0|            0.0|
|550441672312512512|    KeralaGuy77|1420070496|Insanity of today...|          0|          0|       0|   2015-01-01|     00:01:36|               0.0|        -0.8271|
|550441732014223360|    DozenStocks|1420070510|S&P100 #Stocks Pe...|          0|          0|       0|   2015-01-01|     00:01:50|               0.0|        -0.2732|
|550442977

In [21]:
#df_tweets_clean.write.csv('/content/drive/MyDrive/Proyecto integrador/Datasets/Trusted/tweets_cleaned_sentiment.csv')

In [22]:
#df_tweets_cleaned =spark.read.csv('/content/drive/MyDrive/Proyecto integrador/Datasets/Trusted/tweets_cleaned_sentiment.csv',inferSchema=True,header=True,sep=',')
#df_tweets_cleand.printSchema()

# Limpieza y Filtrado de DataFrames

In [23]:
df_aggregated_comments = df_tweets_clean.groupBy("post_date_utc").agg({"comment_num":"sum"}).collect()
df_aggregated_retweets = df_tweets_clean.groupBy("post_date_utc").agg({"retweet_num":"sum"}).collect()
df_aggregated_likes = df_tweets_clean.groupBy("post_date_utc").agg({"like_num":"sum"}).collect()

In [24]:
df_tweets_clean.count()

3717964

In [25]:
df_comments_sum = spark.createDataFrame(df_aggregated_comments)
df_retweets_sum = spark.createDataFrame(df_aggregated_retweets)
df_likes_sum = spark.createDataFrame(df_aggregated_likes)

In [26]:
df_likes_sum.show(10)

+-------------+-------------+
|post_date_utc|sum(like_num)|
+-------------+-------------+
|   2015-03-09|         4122|
|   2015-05-19|          918|
|   2015-03-06|         1479|
|   2015-04-09|         1085|
|   2015-09-02|          947|
|   2015-12-22|         1117|
|   2016-01-28|         4220|
|   2015-05-10|          253|
|   2015-09-28|          958|
|   2015-03-12|          936|
+-------------+-------------+
only showing top 10 rows



In [27]:
exploratory_df1 = df_comments_sum.alias("a").join(df_retweets_sum.alias("b"), df_comments_sum.post_date_utc == df_retweets_sum.post_date_utc,'left').select("a.post_date_utc","a.sum(comment_num)","b.sum(retweet_num)")
exploratory_df1 = exploratory_df1.alias("a").join(df_likes_sum.alias("b"), exploratory_df1.post_date_utc == df_likes_sum.post_date_utc,'left').select("a.post_date_utc","a.sum(comment_num)","a.sum(retweet_num)","b.sum(like_num)")
exploratory_df1.show(10)

+-------------+----------------+----------------+-------------+
|post_date_utc|sum(comment_num)|sum(retweet_num)|sum(like_num)|
+-------------+----------------+----------------+-------------+
|   2015-03-09|            1058|            4705|         4122|
|   2015-05-19|             249|             632|          918|
|   2015-03-06|             353|            1278|         1479|
|   2015-04-09|             288|             881|         1085|
|   2015-09-02|             252|             637|          947|
|   2015-12-22|             143|             560|         1117|
|   2016-01-28|             957|            2333|         4220|
|   2015-05-10|              29|             177|          253|
|   2015-09-28|             216|            1016|          958|
|   2015-03-12|             186|             620|          936|
+-------------+----------------+----------------+-------------+
only showing top 10 rows



In [28]:
#exploratory_df1.write.csv('/content/drive/MyDrive/Proyecto integrador/Datasets/Trusted/exploratory_sentiment_v2.csv')

In [29]:
joined_df = df_tweets_clean.alias("a").join(df_comments_sum.alias("b"), df_tweets_clean.post_date_utc == df_comments_sum.post_date_utc, 'left').select("a.tweet_id","a.writer","a.body","a.comment_num","a.retweet_num","a.like_num","a.post_date_utc","a.post_time_utc","a.textBlob_sentiment","a.vader_sentiment","b.sum(comment_num)")
joined_df = joined_df.alias("a").join(df_retweets_sum.alias("b"), joined_df.post_date_utc == df_retweets_sum.post_date_utc, 'left').select("a.tweet_id","a.writer","a.body","a.comment_num","a.retweet_num","a.like_num","a.post_date_utc","a.post_time_utc","a.textBlob_sentiment","a.vader_sentiment","a.sum(comment_num)","b.sum(retweet_num)")
joined_df = joined_df.alias("a").join(df_likes_sum.alias("b"), joined_df.post_date_utc == df_likes_sum.post_date_utc, 'left').select("a.tweet_id","a.writer","a.body","a.comment_num","a.retweet_num","a.like_num","a.post_date_utc","a.post_time_utc","a.textBlob_sentiment","a.vader_sentiment","a.sum(comment_num)","a.sum(retweet_num)","b.sum(like_num)")

In [30]:
joined_df.show(10)

+------------------+---------------+--------------------+-----------+-----------+--------+-------------+-------------+------------------+---------------+----------------+----------------+-------------+
|          tweet_id|         writer|                body|comment_num|retweet_num|like_num|post_date_utc|post_time_utc|textBlob_sentiment|vader_sentiment|sum(comment_num)|sum(retweet_num)|sum(like_num)|
+------------------+---------------+--------------------+-----------+-----------+--------+-------------+-------------+------------------+---------------+----------------+----------------+-------------+
|550441509175443456|VisualStockRSRC|lx21 made $10,008...|          0|          0|       1|   2015-01-01|     00:00:57|               0.0|            0.0|             120|            1562|          651|
|550441672312512512|    KeralaGuy77|Insanity of today...|          0|          0|       0|   2015-01-01|     00:01:36|               0.0|        -0.8271|             120|            1562|     

In [31]:
joined_df =  joined_df.withColumn("percentage_comment",joined_df["comment_num"]/joined_df["sum(comment_num)"]).withColumn("percentage_retweet",joined_df["retweet_num"]/joined_df["sum(retweet_num)"]).withColumn("percentage_like",joined_df["like_num"]/joined_df["sum(like_num)"])

In [32]:
daily_sentiment_df = joined_df.select("post_date_utc","percentage_like","percentage_comment","percentage_retweet","textBlob_sentiment","vader_sentiment")
daily_sentiment_df.show(10)

+-------------+--------------------+------------------+------------------+------------------+---------------+
|post_date_utc|     percentage_like|percentage_comment|percentage_retweet|textBlob_sentiment|vader_sentiment|
+-------------+--------------------+------------------+------------------+------------------+---------------+
|   2015-01-01|0.001536098310291...|               0.0|               0.0|               0.0|            0.0|
|   2015-01-01|                 0.0|               0.0|               0.0|               0.0|        -0.8271|
|   2015-01-01|                 0.0|               0.0|               0.0|               0.0|        -0.2732|
|   2015-01-01|0.001536098310291...|               0.0|               0.0|              0.25|            0.0|
|   2015-01-01|0.001536098310291...|               0.0|               0.0|               0.0|            0.0|
|   2015-01-01|0.001536098310291...|               0.0|               0.0|               0.0|            0.0|
|   2015-0

In [33]:
daily_sentiment_df = daily_sentiment_df.withColumn("tb_sentiment_agg_like",daily_sentiment_df["percentage_like"]*daily_sentiment_df["textBlob_sentiment"]).withColumn("vader_sentiment_agg_like",daily_sentiment_df["percentage_like"]*daily_sentiment_df["vader_sentiment"])
daily_sentiment_df.show(20)

+-------------+--------------------+------------------+--------------------+------------------+---------------+---------------------+------------------------+
|post_date_utc|     percentage_like|percentage_comment|  percentage_retweet|textBlob_sentiment|vader_sentiment|tb_sentiment_agg_like|vader_sentiment_agg_like|
+-------------+--------------------+------------------+--------------------+------------------+---------------+---------------------+------------------------+
|   2015-01-01|0.001536098310291...|               0.0|                 0.0|               0.0|            0.0|                  0.0|                     0.0|
|   2015-01-01|                 0.0|               0.0|                 0.0|               0.0|        -0.8271|                  0.0|                    -0.0|
|   2015-01-01|                 0.0|               0.0|                 0.0|               0.0|        -0.2732|                  0.0|                    -0.0|
|   2015-01-01|0.001536098310291...|          

In [34]:
daily_sentiment_df = daily_sentiment_df.withColumn("tb_sentiment_agg_comment",daily_sentiment_df["percentage_comment"]*daily_sentiment_df["textBlob_sentiment"]).withColumn("vader_sentiment_agg_comment",daily_sentiment_df["percentage_comment"]*daily_sentiment_df["vader_sentiment"])
daily_sentiment_df = daily_sentiment_df.withColumn("tb_sentiment_agg_retweet",daily_sentiment_df["percentage_retweet"]*daily_sentiment_df["textBlob_sentiment"]).withColumn("vader_sentiment_agg_retweet",daily_sentiment_df["percentage_retweet"]*daily_sentiment_df["vader_sentiment"])
daily_sentiment_df.show(10)

+-------------+--------------------+------------------+------------------+------------------+---------------+---------------------+------------------------+------------------------+---------------------------+------------------------+---------------------------+
|post_date_utc|     percentage_like|percentage_comment|percentage_retweet|textBlob_sentiment|vader_sentiment|tb_sentiment_agg_like|vader_sentiment_agg_like|tb_sentiment_agg_comment|vader_sentiment_agg_comment|tb_sentiment_agg_retweet|vader_sentiment_agg_retweet|
+-------------+--------------------+------------------+------------------+------------------+---------------+---------------------+------------------------+------------------------+---------------------------+------------------------+---------------------------+
|   2015-01-01|0.001536098310291...|               0.0|               0.0|               0.0|            0.0|                  0.0|                     0.0|                     0.0|                        0.0|  

In [35]:
sentiment_exploratory2 = daily_sentiment_df.select("post_date_utc","textBlob_sentiment","vader_sentiment")
sentiment_exploratory2.show(10)

+-------------+------------------+---------------+
|post_date_utc|textBlob_sentiment|vader_sentiment|
+-------------+------------------+---------------+
|   2015-01-01|               0.0|            0.0|
|   2015-01-01|               0.0|        -0.8271|
|   2015-01-01|               0.0|        -0.2732|
|   2015-01-01|              0.25|            0.0|
|   2015-01-01|               0.0|            0.0|
|   2015-01-01|               0.0|            0.0|
|   2015-01-01|               0.0|            0.0|
|   2015-01-01|               0.0|            0.0|
|   2015-01-01|               0.0|            0.0|
|   2015-01-01|               0.0|            0.0|
+-------------+------------------+---------------+
only showing top 10 rows



In [36]:
sentiment_exploratory2 = sentiment_exploratory2.withColumn("textBlob_vector", when(col("textBlob_sentiment") >= 0, 1).otherwise(0)).withColumn("vader_vector", when(col("vader_sentiment") >= 0, 1).otherwise(0))
sentiment_exploratory2.show(20)

+-------------+------------------+---------------+---------------+------------+
|post_date_utc|textBlob_sentiment|vader_sentiment|textBlob_vector|vader_vector|
+-------------+------------------+---------------+---------------+------------+
|   2015-01-01|               0.0|            0.0|              1|           1|
|   2015-01-01|               0.0|        -0.8271|              1|           0|
|   2015-01-01|               0.0|        -0.2732|              1|           0|
|   2015-01-01|              0.25|            0.0|              1|           1|
|   2015-01-01|               0.0|            0.0|              1|           1|
|   2015-01-01|               0.0|            0.0|              1|           1|
|   2015-01-01|               0.0|            0.0|              1|           1|
|   2015-01-01|               0.0|            0.0|              1|           1|
|   2015-01-01|               0.0|            0.0|              1|           1|
|   2015-01-01|               0.0|      

In [37]:
test1 = sentiment_exploratory2.groupBy("post_date_utc").count()
test1.show(10)
#df_tweets_clean.groupBy("post_date_utc").agg({"comment_num":"sum"}).collect()

+-------------+-----+
|post_date_utc|count|
+-------------+-----+
|   2015-03-09| 5579|
|   2015-05-19| 2555|
|   2015-03-06| 2956|
|   2015-04-09| 2150|
|   2015-09-02| 1758|
|   2015-12-22| 1671|
|   2016-01-28| 4483|
|   2015-05-10|  863|
|   2015-09-28| 2820|
|   2015-03-12| 2394|
+-------------+-----+
only showing top 10 rows



In [38]:
test1.show(10)

+-------------+-----+
|post_date_utc|count|
+-------------+-----+
|   2015-03-09| 5579|
|   2015-05-19| 2555|
|   2015-03-06| 2956|
|   2015-04-09| 2150|
|   2015-09-02| 1758|
|   2015-12-22| 1671|
|   2016-01-28| 4483|
|   2015-05-10|  863|
|   2015-09-28| 2820|
|   2015-03-12| 2394|
+-------------+-----+
only showing top 10 rows



In [39]:
tb_sum = sentiment_exploratory2.groupBy("post_date_utc").agg({"textBlob_vector":"sum"}).collect()
tb_sum = spark.createDataFrame(tb_sum)
tb_sum.show(10)

+-------------+--------------------+
|post_date_utc|sum(textBlob_vector)|
+-------------+--------------------+
|   2015-03-09|                4997|
|   2015-05-19|                2242|
|   2015-03-06|                2592|
|   2015-04-09|                1867|
|   2015-09-02|                1577|
|   2015-12-22|                1531|
|   2016-01-28|                3872|
|   2015-05-10|                 764|
|   2015-09-28|                2599|
|   2015-03-12|                2112|
+-------------+--------------------+
only showing top 10 rows



In [40]:
vader_sum = sentiment_exploratory2.groupBy("post_date_utc").agg({"vader_vector":"sum"}).collect()
vader_sum = spark.createDataFrame(vader_sum)
vader_sum.show(10)

+-------------+-----------------+
|post_date_utc|sum(vader_vector)|
+-------------+-----------------+
|   2015-03-09|             4763|
|   2015-05-19|             2096|
|   2015-03-06|             2552|
|   2015-04-09|             1804|
|   2015-09-02|             1533|
|   2015-12-22|             1441|
|   2016-01-28|             3678|
|   2015-05-10|              736|
|   2015-09-28|             2535|
|   2015-03-12|             2034|
+-------------+-----------------+
only showing top 10 rows



In [41]:
sum_sent_df = tb_sum.alias("a").join(vader_sum.alias("b"), tb_sum.post_date_utc == vader_sum.post_date_utc,'left').select("a.post_date_utc","a.sum(textBlob_vector)","b.sum(vader_vector)")
sum_sent_df_total = sum_sent_df.alias("a").join(vader_sum.alias("b"), tb_sum.post_date_utc == vader_sum.post_date_utc,'left').select("a.post_date_utc","a.sum(textBlob_vector)","b.sum(vader_vector)")
sum_sent_df.show(10)

+-------------+--------------------+-----------------+
|post_date_utc|sum(textBlob_vector)|sum(vader_vector)|
+-------------+--------------------+-----------------+
|   2015-03-09|                4997|             4763|
|   2015-05-19|                2242|             2096|
|   2015-03-06|                2592|             2552|
|   2015-04-09|                1867|             1804|
|   2015-09-02|                1577|             1533|
|   2015-12-22|                1531|             1441|
|   2016-01-28|                3872|             3678|
|   2015-05-10|                 764|              736|
|   2015-09-28|                2599|             2535|
|   2015-03-12|                2112|             2034|
+-------------+--------------------+-----------------+
only showing top 10 rows



In [42]:
sum_sent_df.write.csv('/content/drive/MyDrive/Proyecto integrador/Datasets/Trusted/sentimiento_dia_v1.csv')

In [43]:
test1.write.csv('/content/drive/MyDrive/Proyecto integrador/Datasets/Trusted/sentimiento_dia_totales.csv')

In [32]:
pandas = daily_sentiment_df.toPandas()
pandas

NameError: ignored

In [33]:
sentiment_df = pandas.copy()
sentiment_df

NameError: ignored

In [ ]:
with open('/content/drive/MyDrive/Proyecto integrador/Datasets/Raw/tweets_sentimientos_limpios.csv', 'w' , encoding = 'utf-8-sig') as f:
  sentiment_df.to_csv(f)

# **Modelo**

In [1]:
import pandas as pd

sentiment_df = pd.read_csv('/content/drive/MyDrive/Proyecto integrador/Datasets/Trusted/tweets_sentimientos_trusted.csv',low_memory=False)
sentiment_df.head(5)

,Unnamed: 0,Date,tb_sentiment_agg_like,vader_sentiment_agg_like,tb_sentiment_agg_comment,vader_sentiment_agg_comment,tb_sentiment_agg_retweet,vader_sentiment_agg_retweet,tb_likes_vector,tb_comments_vector,tb_retweets_vector,vader_likes_vector,vader_comments_vector,vader_retweets_vector
0,0,2015-01-01,-0.270301,0.010241,-0.411497,-0.155643,0.006467,0.115964,-1,-1,0,0,-1,1
1,1,2015-01-02,0.077530,0.121911,0.053342,0.075837,0.197816,0.125320,1,1,1,1,1,1
2,2,2015-01-03,0.084974,0.135777,0.070563,0.169745,-0.006333,0.099781,1,1,0,1,1,1
3,3,2015-01-04,-0.066925,0.197441,-0.111962,0.117920,-0.253550,0.160401,-1,-1,-1,1,1,1
4,4,2015-01-05,0.104477,0.160160,0.089984,0.132301,0.094667,0.149451,1,1,1,1,1,1


In [2]:
positive_treshold = 0.000000001
negative_treshold = -0.000000001

In [3]:
sentiment_df['tb_likes_vector'] = sentiment_df.apply(lambda row: 1 if row['tb_sentiment_agg_like']>positive_treshold else (0 if row['tb_sentiment_agg_like']<negative_treshold else 0),axis=1)
sentiment_df['tb_comments_vector'] = sentiment_df.apply(lambda row: 1 if row['tb_sentiment_agg_comment']>positive_treshold else (0 if row['tb_sentiment_agg_comment']<negative_treshold else 0),axis=1)
sentiment_df['tb_retweets_vector'] = sentiment_df.apply(lambda row: 1 if row['tb_sentiment_agg_retweet']>positive_treshold else (0 if row['tb_sentiment_agg_retweet']<negative_treshold else 0),axis=1)
sentiment_df['vader_likes_vector'] = sentiment_df.apply(lambda row: 1 if row['vader_sentiment_agg_like']>positive_treshold else (0 if row['vader_sentiment_agg_like']<negative_treshold else 0),axis=1)
sentiment_df['vader_comments_vector'] = sentiment_df.apply(lambda row: 1 if row['vader_sentiment_agg_comment']>positive_treshold else (0 if row['vader_sentiment_agg_comment']<negative_treshold else 0),axis=1)
sentiment_df['vader_retweets_vector'] = sentiment_df.apply(lambda row: 1 if row['vader_sentiment_agg_retweet']>positive_treshold else (0 if row['vader_sentiment_agg_retweet']<negative_treshold else 0),axis=1)
sentiment_df.head(5)

,Unnamed: 0,Date,tb_sentiment_agg_like,vader_sentiment_agg_like,tb_sentiment_agg_comment,vader_sentiment_agg_comment,tb_sentiment_agg_retweet,vader_sentiment_agg_retweet,tb_likes_vector,tb_comments_vector,tb_retweets_vector,vader_likes_vector,vader_comments_vector,vader_retweets_vector
0,0,2015-01-01,-0.270301,0.010241,-0.411497,-0.155643,0.006467,0.115964,0,0,1,1,0,1
1,1,2015-01-02,0.077530,0.121911,0.053342,0.075837,0.197816,0.125320,1,1,1,1,1,1
2,2,2015-01-03,0.084974,0.135777,0.070563,0.169745,-0.006333,0.099781,1,1,0,1,1,1
3,3,2015-01-04,-0.066925,0.197441,-0.111962,0.117920,-0.253550,0.160401,0,0,0,1,1,1
4,4,2015-01-05,0.104477,0.160160,0.089984,0.132301,0.094667,0.149451,1,1,1,1,1,1


In [4]:
sentiment_df.describe()

,Unnamed: 0,tb_sentiment_agg_like,vader_sentiment_agg_like,tb_sentiment_agg_comment,vader_sentiment_agg_comment,tb_sentiment_agg_retweet,vader_sentiment_agg_retweet,tb_likes_vector,tb_comments_vector,tb_retweets_vector,vader_likes_vector,vader_comments_vector,vader_retweets_vector
count,1826.000000,1826.000000,1826.000000,1826.000000,1826.000000,1826.000000,1826.000000,1826.000000,1826.000000,1826.000000,1826.000000,1826.000000,1826.000000
mean,912.500000,0.085804,0.125259,0.075172,0.105969,0.083771,0.118919,0.985214,0.979737,0.967689,0.973165,0.956736,0.947426
std,527.265114,0.041012,0.064003,0.042529,0.067595,0.056683,0.087533,0.120730,0.140937,0.176873,0.161644,0.203506,0.223242
min,0.000000,-0.270301,-0.178860,-0.411497,-0.257721,-0.312821,-0.408984,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,456.250000,0.062905,0.086649,0.052974,0.066890,0.053816,0.070661,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
50%,912.500000,0.083387,0.124544,0.073660,0.105798,0.079594,0.114996,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
75%,1368.750000,0.106846,0.161018,0.097427,0.143910,0.107925,0.165688,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
max,1825.000000,0.318968,0.510215,0.346616,0.528208,0.634680,0.580452,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [5]:
#Correlacion entre mercado y tweets 1dia.
market_data = pd.read_csv('/content/drive/MyDrive/Proyecto integrador/Datasets/Trusted/dataset_limpio_98.csv',low_memory=False)
market_data.head(5)

,Date,Open,High,Low,Close,Adj Close,Volume,rdto
0,2014-09-03,4610.140137,4610.140137,4565.379883,4572.560059,4572.560059,1897450000,-0.557391
1,2014-09-04,4581.520020,4603.149902,4553.310059,4562.290039,4562.290039,1728700000,-0.224601
2,2014-09-05,4560.629883,4583.000000,4542.740234,4582.899902,4582.899902,1641830000,0.451744
3,2014-09-08,4579.060059,4600.399902,4570.229980,4592.290039,4592.290039,1670210000,0.204895
4,2014-09-09,4588.830078,4599.029785,4544.439941,4552.290039,4552.290039,1956550000,-0.871025


In [6]:
market_data.describe()

,Open,High,Low,Close,Adj Close,Volume,rdto
count,1314.000000,1314.000000,1314.000000,1314.000000,1314.000000,1.314000e+03,1314.000000
mean,6183.878285,6214.810772,6150.111072,6185.540842,6185.540842,1.982824e+09,0.074772
std,1300.990371,1306.551724,1294.523970,1301.268769,1301.268769,3.308009e+08,0.927997
min,4133.250000,4231.540039,4116.600098,4213.660156,4213.660156,7.068800e+08,-3.409397
25%,4980.177612,5000.542602,4956.109863,4982.815064,4982.815064,1.773145e+09,-0.361756
50%,5951.689941,5963.155029,5936.010010,5950.299805,5950.299805,1.943725e+09,0.096007
75%,7422.627564,7458.229981,7373.557617,7421.189941,7421.189941,2.153475e+09,0.605463
max,9049.469727,9052.000000,8987.320313,9022.389648,9022.389648,3.359380e+09,3.258700


In [7]:
market_data.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'rdto'], dtype='object')

In [8]:
mkt_th_positive = 0.000001
mkt_th_negative = -0.000001

mkt_th_positive_extreme = 1.2
mkt_th_negative_extreme = -1.2

market_data['vector1d'] = market_data.apply(lambda row: 1 if row['rdto']>mkt_th_positive else(0 if row['rdto']<mkt_th_negative else 0),axis=1)

market_data['vector1d'] = market_data.apply(lambda row: 1 if row['rdto']>mkt_th_positive_extreme else(0 if row['rdto']<mkt_th_negative_extreme else row['vector1d']),axis=1)

market_data.head(10)

,Date,Open,High,Low,Close,Adj Close,Volume,rdto,vector1d
0,2014-09-03,4610.140137,4610.140137,4565.379883,4572.560059,4572.560059,1897450000,-0.557391,0
1,2014-09-04,4581.520020,4603.149902,4553.310059,4562.290039,4562.290039,1728700000,-0.224601,0
2,2014-09-05,4560.629883,4583.000000,4542.740234,4582.899902,4582.899902,1641830000,0.451744,1
3,2014-09-08,4579.060059,4600.399902,4570.229980,4592.290039,4592.290039,1670210000,0.204895,1
4,2014-09-09,4588.830078,4599.029785,4544.439941,4552.290039,4552.290039,1956550000,-0.871025,0
5,2014-09-10,4554.149902,4587.100098,4544.839844,4586.520020,4586.520020,1808650000,0.751929,1
6,2014-09-11,4567.640137,4591.810059,4559.750000,4591.810059,4591.810059,1704850000,0.115339,1
7,2014-09-12,4588.770020,4590.080078,4555.680176,4567.600098,4567.600098,1784170000,-0.527242,0
8,2014-09-15,4567.450195,4567.470215,4506.729980,4518.899902,4518.899902,1940520000,-1.066210,0
9,2014-09-16,4502.109863,4558.240234,4499.870117,4552.759766,4552.759766,1879350000,0.749294,1


In [9]:
market_data.describe()

,Open,High,Low,Close,Adj Close,Volume,rdto,vector1d
count,1314.000000,1314.000000,1314.000000,1314.000000,1314.000000,1.314000e+03,1314.000000,1314.000000
mean,6183.878285,6214.810772,6150.111072,6185.540842,6185.540842,1.982824e+09,0.074772,0.561644
std,1300.990371,1306.551724,1294.523970,1301.268769,1301.268769,3.308009e+08,0.927997,0.496374
min,4133.250000,4231.540039,4116.600098,4213.660156,4213.660156,7.068800e+08,-3.409397,0.000000
25%,4980.177612,5000.542602,4956.109863,4982.815064,4982.815064,1.773145e+09,-0.361756,0.000000
50%,5951.689941,5963.155029,5936.010010,5950.299805,5950.299805,1.943725e+09,0.096007,1.000000
75%,7422.627564,7458.229981,7373.557617,7421.189941,7421.189941,2.153475e+09,0.605463,1.000000
max,9049.469727,9052.000000,8987.320313,9022.389648,9022.389648,3.359380e+09,3.258700,1.000000


In [10]:
mkt_positives = market_data[market_data['vector1d']>0].count()
mkt_positives

Date         738
Open         738
High         738
Low          738
Close        738
Adj Close    738
Volume       738
rdto         738
vector1d     738
dtype: int64

In [11]:
mkt_negatives = market_data[market_data['vector1d']<0].count()
mkt_negatives

Date         0
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
rdto         0
vector1d     0
dtype: int64

In [12]:
mkt_neutral = market_data[market_data['vector1d']==0].count()
mkt_neutral

Date         576
Open         576
High         576
Low          576
Close        576
Adj Close    576
Volume       576
rdto         576
vector1d     576
dtype: int64

In [13]:
market_vectors = market_data[['Date','rdto','vector1d']]
market_vectors

,Date,rdto,vector1d
0,2014-09-03,-0.557391,0
1,2014-09-04,-0.224601,0
2,2014-09-05,0.451744,1
3,2014-09-08,0.204895,1
4,2014-09-09,-0.871025,0
...,...,...,...
1309,2019-12-23,0.231827,1
1310,2019-12-24,0.080816,1
1311,2019-12-26,0.776396,1
1312,2019-12-27,-0.174782,0


In [14]:
market_vectors['rdto_shifted'] = market_vectors['rdto'].shift(-1)
market_vectors['vector1d_shifted'] = market_vectors['vector1d'].shift(-1)
market_vectors

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Date,rdto,vector1d,rdto_shifted,vector1d_shifted
0,2014-09-03,-0.557391,0,-0.224601,0.0
1,2014-09-04,-0.224601,0,0.451744,1.0
2,2014-09-05,0.451744,1,0.204895,1.0
3,2014-09-08,0.204895,1,-0.871025,0.0
4,2014-09-09,-0.871025,0,0.751929,1.0
...,...,...,...,...,...
1309,2019-12-23,0.231827,1,0.080816,1.0
1310,2019-12-24,0.080816,1,0.776396,1.0
1311,2019-12-26,0.776396,1,-0.174782,0.0
1312,2019-12-27,-0.174782,0,-0.673170,0.0


In [15]:
#Juntamos con sentimientos.
sentiment_df.rename(columns={'post_date_utc':'Date'}, inplace = True)
print(sentiment_df.columns)

joined_df = pd.merge(sentiment_df,market_vectors,how='left',on=['Date'])


Index(['Unnamed: 0', 'Date', 'tb_sentiment_agg_like',
       'vader_sentiment_agg_like', 'tb_sentiment_agg_comment',
       'vader_sentiment_agg_comment', 'tb_sentiment_agg_retweet',
       'vader_sentiment_agg_retweet', 'tb_likes_vector', 'tb_comments_vector',
       'tb_retweets_vector', 'vader_likes_vector', 'vader_comments_vector',
       'vader_retweets_vector'],
      dtype='object')


In [16]:
joined_df

,Unnamed: 0,Date,tb_sentiment_agg_like,vader_sentiment_agg_like,tb_sentiment_agg_comment,vader_sentiment_agg_comment,tb_sentiment_agg_retweet,vader_sentiment_agg_retweet,tb_likes_vector,tb_comments_vector,tb_retweets_vector,vader_likes_vector,vader_comments_vector,vader_retweets_vector,rdto,vector1d,rdto_shifted,vector1d_shifted
0,0,2015-01-01,-0.270301,0.010241,-0.411497,-0.155643,0.006467,0.115964,0,0,1,1,0,1,NaN,NaN,NaN,NaN
1,1,2015-01-02,0.077530,0.121911,0.053342,0.075837,0.197816,0.125320,1,1,1,1,1,1,-0.195094,0.0,-1.570620,0.0
2,2,2015-01-03,0.084974,0.135777,0.070563,0.169745,-0.006333,0.099781,1,1,0,1,1,1,NaN,NaN,NaN,NaN
3,3,2015-01-04,-0.066925,0.197441,-0.111962,0.117920,-0.253550,0.160401,0,0,0,1,1,1,NaN,NaN,NaN,NaN
4,4,2015-01-05,0.104477,0.160160,0.089984,0.132301,0.094667,0.149451,1,1,1,1,1,1,-1.570620,0.0,-1.285947,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1821,1821,2019-12-27,0.048672,0.144104,0.043899,0.123508,0.116598,0.254251,1,1,1,1,1,1,-0.174782,0.0,-0.673170,0.0
1822,1822,2019-12-28,0.101040,0.204908,0.067188,0.154332,0.097260,0.179383,1,1,1,1,1,1,NaN,NaN,NaN,NaN
1823,1823,2019-12-29,0.059166,0.099699,0.068911,0.090600,0.120346,0.165431,1,1,1,1,1,1,NaN,NaN,NaN,NaN
1824,1824,2019-12-30,0.108990,0.162796,0.093819,0.098175,0.281425,0.452046,1,1,1,1,1,1,-0.673170,0.0,NaN,NaN


In [17]:
joined_df = joined_df[joined_df['vector1d_shifted'].notna()]
joined_df

,Unnamed: 0,Date,tb_sentiment_agg_like,vader_sentiment_agg_like,tb_sentiment_agg_comment,vader_sentiment_agg_comment,tb_sentiment_agg_retweet,vader_sentiment_agg_retweet,tb_likes_vector,tb_comments_vector,tb_retweets_vector,vader_likes_vector,vader_comments_vector,vader_retweets_vector,rdto,vector1d,rdto_shifted,vector1d_shifted
1,1,2015-01-02,0.077530,0.121911,0.053342,0.075837,0.197816,0.125320,1,1,1,1,1,1,-0.195094,0.0,-1.570620,0.0
4,4,2015-01-05,0.104477,0.160160,0.089984,0.132301,0.094667,0.149451,1,1,1,1,1,1,-1.570620,0.0,-1.285947,0.0
5,5,2015-01-06,0.076349,0.134843,0.067247,0.099407,0.049743,0.048010,1,1,1,1,1,1,-1.285947,0.0,1.256983,1.0
6,6,2015-01-07,0.070802,0.116633,0.020652,0.097276,0.052307,0.053282,1,1,1,1,1,1,1.256983,1.0,1.843249,1.0
7,7,2015-01-08,0.125342,0.143514,0.136843,0.148891,0.118006,0.086948,1,1,1,1,1,1,1.843249,1.0,-0.678185,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1813,1813,2019-12-19,0.084212,0.225914,0.084647,0.012938,0.099074,0.190347,1,1,1,1,1,1,0.673779,1.0,0.231827,1.0
1817,1817,2019-12-23,0.095159,0.202943,0.086354,0.138893,0.104797,0.217780,1,1,1,1,1,1,0.231827,1.0,0.080816,1.0
1818,1818,2019-12-24,0.097582,0.184620,0.098713,0.281355,0.107862,0.248562,1,1,1,1,1,1,0.080816,1.0,0.776396,1.0
1820,1820,2019-12-26,0.122950,0.143449,0.100937,0.132633,0.122595,0.201793,1,1,1,1,1,1,0.776396,1.0,-0.174782,0.0


In [18]:
correlation_df = joined_df[['tb_likes_vector','tb_comments_vector','tb_retweets_vector','vader_likes_vector','vader_comments_vector','vader_retweets_vector','vector1d','vector1d_shifted']]
correlation_df.head(5)

,tb_likes_vector,tb_comments_vector,tb_retweets_vector,vader_likes_vector,vader_comments_vector,vader_retweets_vector,vector1d,vector1d_shifted
1,1,1,1,1,1,1,0.0,0.0
4,1,1,1,1,1,1,0.0,0.0
5,1,1,1,1,1,1,0.0,1.0
6,1,1,1,1,1,1,1.0,1.0
7,1,1,1,1,1,1,1.0,0.0


In [19]:
correlation_df.corr()

,tb_likes_vector,tb_comments_vector,tb_retweets_vector,vader_likes_vector,vader_comments_vector,vader_retweets_vector,vector1d,vector1d_shifted
tb_likes_vector,1.000000,0.175229,0.326446,0.321674,0.042067,0.182601,-0.007187,0.029312
tb_comments_vector,0.175229,1.000000,0.087690,0.047350,0.295542,0.088460,0.012128,-0.004545
tb_retweets_vector,0.326446,0.087690,1.000000,0.128238,0.133694,0.398626,0.009675,-0.000620
vader_likes_vector,0.321674,0.047350,0.128238,1.000000,0.203647,0.345978,-0.013378,-0.001280
vader_comments_vector,0.042067,0.295542,0.133694,0.203647,1.000000,0.260642,0.009675,-0.021210
vader_retweets_vector,0.182601,0.088460,0.398626,0.345978,0.260642,1.000000,0.013869,-0.059920
vector1d,-0.007187,0.012128,0.009675,-0.013378,0.009675,0.013869,1.000000,-0.037214
vector1d_shifted,0.029312,-0.004545,-0.000620,-0.001280,-0.021210,-0.059920,-0.037214,1.000000


In [20]:
from numpy import linalg
import numpy as np
from sklearn.preprocessing import StandardScaler

def PCA(data):
  x_standard = StandardScaler().fit_transform(data)

  s_cov = np.cov(x_standard.T)
  eigenval, eigenvec = np.linalg.eig(s_cov)
  information = eigenval/sum(eigenval)
  print(information*100)
  

In [21]:
PCA(correlation_df)

[25.55254946  5.54342569  7.25412572 14.08905209 13.12184183 12.22736778
 10.90599427 11.30564316]


In [22]:
dataset = joined_df.copy()
dataset = dataset[['rdto','rdto_shifted','tb_likes_vector','tb_comments_vector','tb_retweets_vector','vader_likes_vector','vader_comments_vector','vader_retweets_vector','vector1d','vector1d_shifted']]
dataset

,rdto,rdto_shifted,tb_likes_vector,tb_comments_vector,tb_retweets_vector,vader_likes_vector,vader_comments_vector,vader_retweets_vector,vector1d,vector1d_shifted
1,-0.195094,-1.570620,1,1,1,1,1,1,0.0,0.0
4,-1.570620,-1.285947,1,1,1,1,1,1,0.0,0.0
5,-1.285947,1.256983,1,1,1,1,1,1,0.0,1.0
6,1.256983,1.843249,1,1,1,1,1,1,1.0,1.0
7,1.843249,-0.678185,1,1,1,1,1,1,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...
1813,0.673779,0.231827,1,1,1,1,1,1,1.0,1.0
1817,0.231827,0.080816,1,1,1,1,1,1,1.0,1.0
1818,0.080816,0.776396,1,1,1,1,1,1,1.0,1.0
1820,0.776396,-0.174782,1,1,1,1,1,1,1.0,0.0


In [106]:
dataset['rdto-sinmedia'] = dataset['rdto']-dataset['rdto'].mean()
dataset

,index,rdto,rdto_shifted,tb_likes_vector,tb_comments_vector,tb_retweets_vector,vader_likes_vector,vader_comments_vector,vader_retweets_vector,vector1d,vector1d_shifted,rdto-sinmedia
0,1,-0.195094,-1.570620,1,1,1,1,1,1,0.0,0.0,-0.272870
1,4,-1.570620,-1.285947,1,1,1,1,1,1,0.0,0.0,-1.648397
2,5,-1.285947,1.256983,1,1,1,1,1,1,0.0,1.0,-1.363724
3,6,1.256983,1.843249,1,1,1,1,1,1,1.0,1.0,1.179207
4,7,1.843249,-0.678185,1,1,1,1,1,1,1.0,0.0,1.765472
...,...,...,...,...,...,...,...,...,...,...,...,...
1224,1813,0.673779,0.231827,1,1,1,1,1,1,1.0,1.0,0.596003
1225,1817,0.231827,0.080816,1,1,1,1,1,1,1.0,1.0,0.154050
1226,1818,0.080816,0.776396,1,1,1,1,1,1,1.0,1.0,0.003039
1227,1820,0.776396,-0.174782,1,1,1,1,1,1,1.0,0.0,0.698619


In [108]:
dataset['vector1d-nomedia'] = dataset.apply(lambda x: 0 if x['rdto-sinmedia']<0 else 1, axis=1)
dataset['vector2d'] = dataset.apply(lambda x: 0 if x['rdto-sinmedia']<0 else 1, axis=1)
dataset

,index,rdto,rdto_shifted,tb_likes_vector,tb_comments_vector,tb_retweets_vector,vader_likes_vector,vader_comments_vector,vader_retweets_vector,vector1d,vector1d_shifted,rdto-sinmedia,vector1d-nomedia
0,1,-0.195094,-1.570620,1,1,1,1,1,1,0.0,0.0,-0.272870,0
1,4,-1.570620,-1.285947,1,1,1,1,1,1,0.0,0.0,-1.648397,0
2,5,-1.285947,1.256983,1,1,1,1,1,1,0.0,1.0,-1.363724,0
3,6,1.256983,1.843249,1,1,1,1,1,1,1.0,1.0,1.179207,1
4,7,1.843249,-0.678185,1,1,1,1,1,1,1.0,0.0,1.765472,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1224,1813,0.673779,0.231827,1,1,1,1,1,1,1.0,1.0,0.596003,1
1225,1817,0.231827,0.080816,1,1,1,1,1,1,1.0,1.0,0.154050,1
1226,1818,0.080816,0.776396,1,1,1,1,1,1,1.0,1.0,0.003039,1
1227,1820,0.776396,-0.174782,1,1,1,1,1,1,1.0,0.0,0.698619,1


In [118]:
dataset[dataset['vector1d']==1].count()


index                    689
rdto                     689
rdto_shifted             689
tb_likes_vector          689
tb_comments_vector       689
tb_retweets_vector       689
vader_likes_vector       689
vader_comments_vector    689
vader_retweets_vector    689
vector1d                 689
vector1d_shifted         689
rdto-sinmedia            689
vector1d-nomedia         689
dtype: int64

In [121]:
dataset[dataset['vector1d']==1].count()/len(dataset.index)

index                    0.560618
rdto                     0.560618
rdto_shifted             0.560618
tb_likes_vector          0.560618
tb_comments_vector       0.560618
tb_retweets_vector       0.560618
vader_likes_vector       0.560618
vader_comments_vector    0.560618
vader_retweets_vector    0.560618
vector1d                 0.560618
vector1d_shifted         0.560618
rdto-sinmedia            0.560618
vector1d-nomedia         0.560618
dtype: float64

In [ ]:
dataset[dataset['vector1d']==1].count()/len(dataset.index)

In [23]:
dataset_input = dataset[['tb_likes_vector','vader_likes_vector','vader_comments_vector','vader_retweets_vector','vector1d']]
dataset_input

,tb_likes_vector,vader_likes_vector,vader_comments_vector,vader_retweets_vector,vector1d
1,1,1,1,1,0.0
4,1,1,1,1,0.0
5,1,1,1,1,0.0
6,1,1,1,1,1.0
7,1,1,1,1,1.0
...,...,...,...,...,...
1813,1,1,1,1,1.0
1817,1,1,1,1,1.0
1818,1,1,1,1,1.0
1820,1,1,1,1,1.0


In [24]:
dataset.reset_index(inplace=True)
PCA(dataset_input)

[34.05214992 12.16856243 14.04440519 19.50021348 20.23466897]


In [109]:
#Single Day Solution

from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score

Y = dataset['vector1d_shifted']
X = dataset


vector_accuracy = []
vector_returns = []
vector_returns2 = []


for i in range(100):
  X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

  X_test_return = X_test[['rdto','rdto_shifted']]


  X_train = X_train[['vector1d-nomedia']]
  X_test = X_test[['vector1d-nomedia']]

  model = svm.SVC()
  model = model.fit(X_train,y_train)
  y_pred = model.predict(X_test)

  vector_accuracy.append(accuracy_score(y_test, y_pred))

  def f(x):
    y = x
    #if(x == 0): y=-1 
    #if(x == -2): y=2
    return y
  
  vector_side = ([f(x) for x in y_pred])
  average_daily_return = vector_side * X_test_return['rdto']
  vector_returns.append(((1+(sum(average_daily_return)/len(average_daily_return))/100)**240-1)*100)

  average_daily_return = average_daily_return.tolist()

  dv = 1
  for i in range(len(average_daily_return)):
    dv = dv * (1+(average_daily_return[i]/100))
  
  dv = (dv**(1/len(average_daily_return)))-1

  vector_returns2.append((((1+dv)**240)-1)*100)



print(sum(vector_accuracy)/len(vector_accuracy))
print(sum(vector_returns)/len(vector_returns))
print(sum(vector_returns2)/len(vector_returns2))

0.5613821138211383
20.528421293353507
19.298957994238652


In [136]:
#Single Day Solution with Logistic Reg

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

Y = dataset['vector1d_shifted']
X = dataset


vector_accuracy = []
vector_returns = []
vector_returns2 = []


for i in range(100):
  X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

  X_test_return = X_test[['rdto','rdto_shifted']]


  X_train = X_train[['vector1d-nomedia','tb_likes_vector','vader_retweets_vector','vader_comments_vector']]
  X_test = X_test[['vector1d-nomedia','tb_likes_vector','vader_retweets_vector','vader_comments_vector']]

  model = LogisticRegression()
  model = model.fit(X_train,y_train)
  y_pred = model.predict(X_test)

  vector_accuracy.append(accuracy_score(y_test, y_pred))

  def f(x):
    y = x
    #if(x == 0): y=-1 
    #if(x == -2): y=2
    return y
  
  vector_side = ([f(x) for x in y_pred])
  average_daily_return = vector_side * X_test_return['rdto']
  vector_returns.append(((1+(sum(average_daily_return)/len(average_daily_return))/100)**240-1)*100)

  average_daily_return = average_daily_return.tolist()

  dv = 1
  for i in range(len(average_daily_return)):
    dv = dv * (1+(average_daily_return[i]/100))
  
  dv = (dv**(1/len(average_daily_return)))-1

  vector_returns2.append((((1+dv)**240)-1)*100)



print(sum(vector_accuracy)/len(vector_accuracy))
print(sum(vector_returns)/len(vector_returns))
print(sum(vector_returns2)/len(vector_returns2))

0.5590243902439023
20.87783731120015
19.6557595085344


In [137]:
model.predict_proba(X_test)

array([[0.42456567, 0.57543433],
       [0.42456567, 0.57543433],
       [0.45679602, 0.54320398],
       [0.42456567, 0.57543433],
       [0.42456567, 0.57543433],
       [0.45679602, 0.54320398],
       [0.45679602, 0.54320398],
       [0.45679602, 0.54320398],
       [0.39776203, 0.60223797],
       [0.42456567, 0.57543433],
       [0.42456567, 0.57543433],
       [0.45679602, 0.54320398],
       [0.45679602, 0.54320398],
       [0.45679602, 0.54320398],
       [0.42456567, 0.57543433],
       [0.42456567, 0.57543433],
       [0.42456567, 0.57543433],
       [0.45679602, 0.54320398],
       [0.42456567, 0.57543433],
       [0.42456567, 0.57543433],
       [0.42456567, 0.57543433],
       [0.45679602, 0.54320398],
       [0.45679602, 0.54320398],
       [0.42456567, 0.57543433],
       [0.45679602, 0.54320398],
       [0.45679602, 0.54320398],
       [0.45679602, 0.54320398],
       [0.42456567, 0.57543433],
       [0.42456567, 0.57543433],
       [0.42456567, 0.57543433],
       [0.

In [114]:
y_train

50      1.0
634     0.0
907     1.0
1103    0.0
762     0.0
       ... 
34      1.0
538     0.0
217     1.0
367     1.0
509     0.0
Name: vector1d_shifted, Length: 983, dtype: float64

In [32]:
#Multiple Day Solution

from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score

Y = dataset['vector1d_shifted'].values
X = dataset.values


vector_accuracy = []
vector_returns = []
vector_returns2 = []

lag = 30
X_historic = []
Y_historic = []


for i in range(lag,len(dataset)-lag,1):
  X_historic.append(X[i:i+lag])
  Y_historic.append(Y[i+lag])


for i in range(1):
  X_train, X_test, y_train, y_test = train_test_split(X_historic, Y_historic, test_size=0.3)

  #X_test_return = X_test[['rdto','rdto_shifted']]
  print(X_test[0])

  X_test_return = X_test.copy()
  X_test_return = X_test_return[:,2]
  print(X_test_return)

  X_train = X_train[['vector1d']]
  #X_test = X_test[['vector1d']]

  model = svm.SVC()
  model = model.fit(X_train,y_train)
  y_pred = model.predict(X_test)

  vector_accuracy.append(accuracy_score(y_test, y_pred))

  def f(x):
    y = x
    #if(x == 0): y=-1 
    #if(x == -2): y=2
    return y
  
  vector_side = ([f(x) for x in y_pred])
  average_daily_return = vector_side * X_test_return['rdto']
  vector_returns.append(((1+(sum(average_daily_return)/len(average_daily_return))/100)**240-1)*100)

  average_daily_return = average_daily_return.tolist()

  dv = 1
  for i in range(len(average_daily_return)):
    dv = dv * (1+(average_daily_return[i]/100))
  
  dv = (dv**(1/len(average_daily_return)))-1

  vector_returns2.append((((1+dv)**240)-1)*100)



print(sum(vector_accuracy)/len(vector_accuracy))
print(sum(vector_returns)/len(vector_returns))
print(sum(vector_returns2)/len(vector_returns2))



[[ 4.07000000e+02  1.65625907e+00  2.26974002e+00  1.00000000e+00
   1.00000000e+00  1.00000000e+00  1.00000000e+00  1.00000000e+00
   1.00000000e+00  1.00000000e+00  1.00000000e+00]
 [ 4.11000000e+02  2.26974002e+00  2.21147393e+00  1.00000000e+00
   1.00000000e+00  1.00000000e+00  1.00000000e+00  1.00000000e+00
   1.00000000e+00  1.00000000e+00  1.00000000e+00]
 [ 4.12000000e+02  2.21147393e+00 -1.02601243e+00  1.00000000e+00
   1.00000000e+00  1.00000000e+00  1.00000000e+00  1.00000000e+00
   1.00000000e+00  1.00000000e+00  0.00000000e+00]
 [ 4.13000000e+02 -1.02601243e+00  3.76378525e-01  1.00000000e+00
   1.00000000e+00  1.00000000e+00  1.00000000e+00  1.00000000e+00
   1.00000000e+00  0.00000000e+00  1.00000000e+00]
 [ 4.14000000e+02  3.76378525e-01  1.46921329e+00  1.00000000e+00
   1.00000000e+00  1.00000000e+00  1.00000000e+00  0.00000000e+00
   1.00000000e+00  1.00000000e+00  1.00000000e+00]
 [ 4.17000000e+02  1.46921329e+00 -1.46653919e+00  1.00000000e+00
   1.00000000e+00  

TypeError: ignored

In [ ]:
type(X_train)

In [ ]:
X_train[['vector1d']].sum()

In [ ]:
473/860

In [ ]:
X_test[['vector1d']].count()

In [ ]:
X_test[['vector1d']].sum()

In [ ]:
216/369